<a href="https://colab.research.google.com/github/eddowding/em_colab/blob/main/esmColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',1200)

In [ ]:
Project_id = "ecosystemmarketplace"
SourceDatasetName = "em_07May" 
SourceTableName = "08may_08may" 

ETL(Project_id,SourceDatasetName,SourceTableName)

In [ ]:
def ETL(Project_id,SourceDatasetName,SourceTableName):
  client = bigquery.Client(project=Project_id)

  SaveSrc(client,Project_id,SourceDatasetName,SourceTableName,'esm_Stage','em_src')

  Countries('esm_FINAL','worldcountries')

  SourceDatasetName='esm_Stage'
  SourceTableName = 'em_src'

  CreateRespondents(client,Project_id)
  print('Respondents created')
  CreateBuyers(client,Project_id)
  print('Buyers created')
  CreateProjects(client,Project_id)
  print('Projects created')
  CreateTrades(client,Project_id)
  print('Trades created')

In [ ]:
def SaveSrc(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):

   # em_src
  Query = 'SELECT ROW_NUMBER() OVER (ORDER BY __hevo_id) src_Id,* FROM `'
  Query = Query+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'

  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')

In [ ]:
def CreateRespondentsStage(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):
# creating respondents_Stage
  source_parentId = 'src_Id'
  identifier_column ='respondent_name'
  source_columns = 'RANK() over (Order by trim(em_respondent)) respondent_id , trim(em_respondent) as respondent_name,respondent_role,respondent_profit_status,respondent_website'
  
  Query = 'SELECT '+source_parentId+',' + source_columns +' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'

  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')



In [ ]:
def CreateRespondentsTRN(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):
  
  # creating respondents_TRN
  source_columns = 'respondents.respondent_id,respondents.respondent_name,respondents.respondent_role,countries.id country_id,respondents.respondent_profit_status,respondents.respondent_website '
  JoiningCondition = '`ecosystemmarketplace.esm_Stage.respondents` as respondents LEFT JOIN `ecosystemmarketplace.esm_Stage.em_src` as src ON respondents.src_Id = src.src_Id LEFT JOIN `ecosystemmarketplace.esm_FINAL.worldcountries` as countries ON src.respondent_country = countries.name'
  Query = 'SELECT ' + source_columns +' FROM '+JoiningCondition

  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')


In [ ]:
def CreateRespondentsFINAL(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):

  # creating respondents_FINAL

  source_columns = 'respondent_id,respondent_name,country_id,respondent_profit_status,respondent_website'
  JoiningCondition = '(SELECT row_number() over (Partition by respondent_id order by respondent_id) as entry_number,'+source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'` ) '

  Query = 'SELECT ' + source_columns +' FROM '+JoiningCondition+' where entry_number=1'
  LogQuery = 'SELECT ' + source_columns +' FROM '+JoiningCondition+' where entry_number>1'


  df = client.query(Query).to_dataframe()
  df_Log = client.query(LogQuery).to_dataframe() 

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')
  df_Log.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.Log_'+DestinationTable,if_exists='replace',location='europe-north1')  # From TRN to FINAL




In [ ]:
def CreateRespondents(client,Project_id):
  
  CreateRespondentsStage(client,Project_id,'esm_Stage','em_src','esm_Stage','respondents')
  CreateRespondentsTRN(client,Project_id,'esm_Stage','respondents','esm_TRN','respondents')
  CreateRespondentsFINAL(client,Project_id,'esm_TRN','respondents','esm_FINAL','respondents')

  CreateRoles(client,Project_id,'esm_FINAL','roles')
  CreateRespondentRoles(client,Project_id,'esm_FINAL','respondentroles')


In [ ]:
def CreateBuyersStage(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):
  
  source_parentId = 'src_Id'
  identifier_column ='buyer_name'
  source_columns = 'RANK() over (Order by trim(buyer_name)) buyer_id , trim(buyer_name) as buyer_name,buyer_profit_status,buyer_profit_status_description,buyer_sector'
  Filter = ' WHERE NULLIF(trim('+identifier_column+'),\'\') IS NOT NULL'
  Query = 'SELECT '+source_parentId+',' + source_columns +' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'+Filter

  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')




In [ ]:
def CreateBuyersTRN(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):
  
# creating buyer_TRN
  source_columns = 'buyers.buyer_id,buyers.buyer_name,countries.id country_id,buyers.buyer_profit_status,buyers.buyer_profit_status_description,buyers.buyer_sector'
  JoiningCondition = '`ecosystemmarketplace.esm_Stage.buyers` as buyers LEFT JOIN `ecosystemmarketplace.esm_Stage.em_src` as src ON buyers.src_Id = src.src_Id LEFT JOIN `ecosystemmarketplace.esm_FINAL.worldcountries` as countries ON src.buyer_country = countries.name'
  Query = 'SELECT ' + source_columns +' FROM '+JoiningCondition

  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')



In [ ]:
def CreateBuyersFINAL(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):

# creating buyer_FINAL
  source_columns = 'buyer_id,buyer_name,country_id,buyer_profit_status,buyer_profit_status_description,buyer_sector'
  id_column = 'buyer_id'
  Entry_number = '(SELECT row_number() over (Partition by '+id_column+' order by '+id_column+') as entry_number,'+source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'
  Entry_number_ctd = ') where entry_number'

  Query = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'=1'
  LogQuery = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'>1'


  df = client.query(Query).to_dataframe()
  df_Log = client.query(LogQuery).to_dataframe() 

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')
  df_Log.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.Log_'+DestinationTable,if_exists='replace',location='europe-north1')  # From TRN to FINAL




In [ ]:
def CreateBuyers(client,Project_id):
  CreateBuyersStage(client,Project_id,'esm_Stage','em_src','esm_Stage','buyers')
  CreateBuyersTRN(client,Project_id,'esm_Stage','buyers','esm_TRN','buyers')
  CreateBuyersFINAL(client,Project_id,'esm_TRN','buyers','esm_FINAL','buyers')

In [ ]:
def CreateProjectsStage(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):
  #creating projects_Stage
  source_parentId = 'src_Id'
  source_columns = 'registry_id,project_name,project_s_type,project_category,reduction_removal,third_party_standard,co_benefits_standard,SDG,corsia_eligibility'
  Filter = ' WHERE project_s_type is not null or project_category is not null or reduction_removal is not null or third_party_standard is not null or co_benefits_standard is not null or SDG is not null or corsia_eligibility  is not null'
  Query = 'SELECT '+source_parentId+',' + source_columns +' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'+Filter

  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')



In [ ]:
def CreateProjectsTRN(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):

  source_columns = 'CASE WHEN trim(projects.registry_id) IS NULL THEN CAST(RANK() OVER(ORDER BY projects.src_Id) as string) ELSE trim(projects.registry_id) END project_id,projects.src_Id,projects.project_name em_project_name,projects.project_s_type,projects.project_category,projects.reduction_removal em_reduction_removal,projects.third_party_standard,projects.co_benefits_standard,projects.SDG,countries.id country_id, projects.corsia_eligibility,category.categoryid,emcluster.clusterid,type.typeid'
  project_From =' FROM `ecosystemmarketplace.esm_Stage.projects` as projects LEFT JOIN `ecosystemmarketplace.esm_Stage.em_src` as src ON projects.src_Id = src.src_Id LEFT JOIN `ecosystemmarketplace.esm_FINAL.worldcountries` as countries ON src.project_s_country = countries.name LEFT JOIN `ecosystemmarketplace.esm_FINAL.category` as category ON src.project_category = category.categoryname LEFT JOIN `ecosystemmarketplace.esm_FINAL.cluster` as emcluster ON src.project_cluster = emcluster.clustername LEFT JOIN `ecosystemmarketplace.esm_FINAL.type` as type ON src.project_cluster = type.typename'
  filters = ' where projects.src_Id is not null or projects.project_name is not null or projects.project_s_type is not null or projects.project_category is not null or projects.reduction_removal is not null or projects.third_party_standard is not null or projects.co_benefits_standard is not null or projects.SDG is not null or projects.corsia_eligibility is not null'
  Query = 'SELECT '+source_columns +project_From + filters

  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')


In [ ]:
def CreateProjectsFINAL(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):

  source_columns = 'project_id,em_project_name,project_s_type,project_category,em_reduction_removal,third_party_standard,co_benefits_standard,SDG,country_id, corsia_eligibility,categoryid,clusterid,typeid'
  id_column = 'project_id'
  Entry_number = '(SELECT row_number() over (Partition by '+id_column+' order by '+id_column+') as entry_number,'+source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'
  Entry_number_ctd = ') where entry_number'

  Query = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'=1'
  LogQuery = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'>1'


  df = client.query(Query).to_dataframe()
  df_Log = client.query(LogQuery).to_dataframe() 

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')
  df_Log.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.Log_'+DestinationTable,if_exists='replace',location='europe-north1')  # From TRN to FINAL




In [ ]:
def CreateProjects(client,Project_id):
  
  CreateProjectCategories(client,Project_id)
  CreateProjectType(client,Project_id)
  CreateProjectCluster(client,Project_id)

  CreateProjectsStage(client,Project_id,'esm_Stage','em_src','esm_Stage','projects')
  CreateProjectsTRN(client,Project_id,'esm_Stage','projects','esm_TRN','projects')
  CreateProjectsFINAL(client,Project_id,'esm_TRN','projects','esm_FINAL','projects')

  CreateSDGSMapping(client,Project_id,'esm_FINAL','projects')
  

In [ ]:
def Countries(DestinationDatasetName,DestinationTable):
  DestinationDatasetName = 'esm_FINAL'
  DestinationTable = 'worldcountries'


  Query = '''
  select *
        ,case when 
        name in ('Australia','Austria','Belarus','Belgium','Bulgaria','Canada','Croatia','Cyprus','Czechia','Denmark','Estonia','European Union','Finland','France','Germany','Greece','Hungary','Iceland','Ireland','Italy','Japan','Latvia','Liechtenstein','Lithuania','Luxembourg','Malta','Monaco','Netherlands','New Zealand','Norway','Poland','Portugal','Romania','Russian Federation','Slovakia','Slovenia','Spain','Sweden','Switzerland','Turkey','Ukraine','United Kingdom of Great Britain and Northern Ireland','United States of America') then "Annex1" 
        when name in ('Afghanistan','Albania','Algeria','Andorra','Angola','Antigua and Barbuda','Argentina','Armenia','Azerbaijan','Bahamas','Bahrain','Bangladesh','Barbados','Belize','Benin','Bhutan','Bolivia (Plurinational State of)','Bosnia and Herzegovina','Botswana','Brazil','Brunei Darussalam','Burkina Faso','Burundi','Cabo Verde','Cambodia','Cameroon','Central African Republic','Chad','Chile','China','Colombia','Comoros','Congo','Cook Islands','Costa Rica',"Côte d'Ivoire",'Cuba',"Democratic People's Republic of Korea",'Democratic Republic of the Congo','Djibouti','Dominica','Dominican Republic','Ecuador','Egypt','El Salvador','Equatorial Guinea','Eritrea','Eswatini','Ethiopia','Fiji','Gabon','Gambia','Georgia','Ghana','Grenada','Guatemala','Guinea','Guinea-Bissau','Guyana','Haiti','Honduras','India','Indonesia','Iran (Islamic Republic of)','Iraq','Israel','Jamaica','Jordan','Kazakhstan','Kenya','Kiribati','Kuwait','Kyrgyzstan',"Lao People's Democratic Republic",'Lebanon','Lesotho','Liberia','Libya','Madagascar','Malawi','Malaysia','Maldives','Mali','Marshall Islands','Mauritania','Mauritius','Mexico','Micronesia (Federated States of)','Mongolia','Montenegro','Morocco','Mozambique','Myanmar','Namibia','Nauru','Nepal','Nicaragua','Niger','Nigeria','Niue','North Macedonia','Oman','Pakistan','Palau','Panama','Papua New Guinea','Paraguay','Peru','Philippines','Qatar','Republic of Korea','Republic of Moldova','Rwanda','Saint Kitts and Nevis','Saint Lucia','Saint Vincent and the Grenadines','Samoa','San Marino','Sao Tome and Principe','Saudi Arabia','Senegal','Serbia','Seychelles','Sierra Leone','Singapore','Solomon Islands','Somalia','South Africa','South Sudan','Sri Lanka','State of Palestine','Sudan','Suriname','Syrian Arab Republic','Tajikistan','Thailand','Timor-Leste','Togo','Tonga','Trinidad and Tobago','Tunisia','Turkmenistan','Tuvalu','Uganda','United Arab Emirates','United Republic of Tanzania','Uruguay','Uzbekistan','Vanuatu','Venezuela (Bolivarian Republic of)','Viet Nam','Yemen','Zambia','Zimbabwe') then "Annex2"
        else null
        end Annex
 from `ecosystemmarketplace.esm_Stage.allcountries`
 '''

  tempdf = client.query(Query).to_dataframe()
  tempdf.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')


In [ ]:
def CreateProjectType(client,Project_id):
  DestinationDatasetName = 'esm_FINAL'
  DestinationTable = 'type'


  Query = '''
SELECT row_number() over (order by project_s_type)typeid
        ,project_s_type typename
from
(Select distinct project_s_type
from `ecosystemmarketplace.esm_Stage.em_src`
where project_s_type is not null
order by project_s_type
)
 '''

  tempdf = client.query(Query).to_dataframe()
  tempdf.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')


In [ ]:
def CreateProjectCluster(client,Project_id):
  DestinationDatasetName = 'esm_FINAL'
  DestinationTable = 'cluster'


  Query = '''
SELECT row_number() over (order by project_cluster)clusterid
        ,project_cluster clustername
from
(Select distinct project_cluster
from `ecosystemmarketplace.esm_Stage.em_src`
where project_cluster is not null
order by project_cluster
)
 '''

  tempdf = client.query(Query).to_dataframe()
  tempdf.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')


In [ ]:
def CreateProjectCategories(client,Project_id):
  DestinationDatasetName = 'esm_FINAL'
  DestinationTable = 'category'


  Query = '''
  SELECT row_number() over (order by project_category)categoryid
        ,project_category categoryname
from
(Select distinct project_category
from `ecosystemmarketplace.esm_Stage.em_src`
where project_category is not null
order by project_category
)
  '''

  tempdf = client.query(Query).to_dataframe()
  tempdf.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')


In [ ]:
def CreateRoles(client,Project_id,DestinationDatasetName,DestinationTable):
 

  Query = '''
  SELECT row_number() over (order by respondentroles) roleid
          ,respondentroles as rolename
  from
  (SELECT distinct upper(trim(REPLACE(REPLACE(splittedroles, '\\r', ' '), '\\n', ' '))) respondentroles
  from `ecosystemmarketplace.esm_Stage.em_src`,
  UNNEST(SPLIT(respondent_role)) as splittedroles
  where splittedroles<>''
  order by respondentroles
  )
  '''

  tempdf = client.query(Query).to_dataframe()
  tempdf.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')


In [ ]:
def convertToRoles(RolesNamelist,roles_dict):
  roles = []
  if RolesNamelist is None:
    return None
  else:
    for item in RolesNamelist.split(','):
      if len(item)>0:
        roles.append(roles_dict[item.strip().upper().replace('\r', ' ').replace('\n', ' ')])     
  return roles

In [ ]:
def CreateRespondentRoles(client,Project_id,DestinationDatasetName,DestinationTable):

    # respondentroles
  Query = 'SELECT * FROM `'+Project_id+'.esm_FINAL.roles`'
  roles_df = client.query(Query).to_dataframe()
  roles_dict = roles_df.to_dict()['rolename']
  roles_dict = {v: k for k, v in roles_dict.items()} # reversing key and value

  # Roles esm_FINAL.respondentroles
  Query = 'SELECT * FROM `'+Project_id+'.esm_FINAL.respondents`'
  df = client.query(Query).to_dataframe()

  df_roles = df[['respondent_id','respondent_role']].copy()
  df_roles['respondent_role'] = df_roles['respondent_role'].str.strip()
  df_roles['respondent_role'] = df_roles['respondent_role'].apply(convertToRoles,roles_dict = roles_dict)


  respondent_list=[]
  roles_list=[]


  for rownum,row in enumerate(df_roles.iterrows()):
      respondent = row[1]['respondent_id']

      for index,rolevalue in enumerate(row[1].explode()):
        if index > 0:
          respondent_list.append(respondent)
          roles_list.append(rolevalue)
      
    
  respondentsroles_merged = list(zip(respondent_list,roles_list))
  respondentroles = pd.DataFrame(respondentsroles_merged, columns = ['respondent_id', 'roleid'])

  respondentroles = respondentroles.drop_duplicates()
  respondentroles['roleid'] = respondentroles['roleid'].replace(np.NAN,0).astype('int64')

  respondentroles.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')


# Move this line out of function when finalized.
  # # removing respondent_role from parent table.
  # df = df.drop(axis =1,columns=['respondent_role'])

In [ ]:
def CreateSDGSMapping(client,Project_id,SourceDatasetName,SourceTableName):

  source_columns = 'project_id,SDG'
  filter = " where SDG is not null and SDG NOT LIKE '%UTC%' and SDG not like '%;%' and SDG NOT LIKE '%Not%'"
  Query = 'SELECT ' + source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'+filter

 
  df_sdgs = client.query(Query).to_dataframe()


  project_list=[]
  sdgs_list=[]
  invalid_project_list = []
  invalid_sdgs_list = []

  df_sdgs['SDG'] = df_sdgs['SDG'].str.strip().str.replace(', ',',').str.replace(' ',',').str.split(',')

  for rownum,row in enumerate(df_sdgs.iterrows()):
    project = row[1]['project_id']
    
    for index,sdgvalue in enumerate(row[1].explode()):
      if index > 0:
        if float(sdgvalue)<=17:
          project_list.append(project)
          sdgs_list.append(int(float(sdgvalue)))
        else:
          invalid_project_list.append(project)
          invalid_sdgs_list.append(sdgvalue)          
  projectsdgs_merged = list(zip(project_list,sdgs_list))
  projectsdgs = pd.DataFrame(projectsdgs_merged, columns = ['project_id', 'sdgs'])

  invalidprojectsdgs_merged = list(zip(invalid_project_list,invalid_sdgs_list))
  invalidprojectsdgs = pd.DataFrame(invalidprojectsdgs_merged, columns = ['project_id', 'sdgs'])


  projectsdgs = projectsdgs.rename(columns={'SDG':'sdgs'})
  


  invalidprojectsdgs.to_gbq(project_id=Project_id,destination_table='esm_Stage.invalid_esmprojectsdgs',if_exists='replace',location='europe-north1')
  projectsdgs.to_gbq(project_id=Project_id,destination_table='esm_FINAL.esmprojectsdgs',if_exists='replace',location='europe-north1')



In [ ]:
def CreateRespondentRoles(df):
    # respondentroles
  Query = 'SELECT * FROM `'+Project_id+'.esm_FINAL.roles`'
  roles_df = client.query(Query).to_dataframe()
  roles_dict = roles_df.to_dict()['rolename']
  roles_dict = {v: k for k, v in roles_dict.items()} # reversing key and value

  df_roles = df[['respondent_id','respondent_role']].copy()
  df_roles['respondent_role'] = df_roles['respondent_role'].str.strip()
  df_roles['respondent_role'] = df_roles['respondent_role'].apply(convertToRoles,roles_dict = roles_dict)


  respondent_list=[]
  roles_list=[]


  for rownum,row in enumerate(df_roles.iterrows()):
      respondent = row[1]['respondent_id']

      for index,rolevalue in enumerate(row[1].explode()):
        if index > 0:
          respondent_list.append(respondent)
          roles_list.append(rolevalue)
      
    
  respondentsroles_merged = list(zip(respondent_list,roles_list))
  respondentroles = pd.DataFrame(respondentsroles_merged, columns = ['respondent_id', 'roleid'])

  respondentroles = respondentroles.drop_duplicates()
  respondentroles['roleid'] = respondentroles['roleid'].replace(np.NAN,0).astype('int64')


  return respondentroles

# Move this line out of function when finalized.
  # # removing respondent_role from parent table.
  # df = df.drop(axis =1,columns=['respondent_role'])

In [ ]:
def CreateTradesStage(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):
  source_columns = 'src_Id trade_id,year_reported,registry_id,volume_transacted_tco2e,volume_w_price,value trade_total_value_USD,conversion,average_price_usd_tco2e price_USD_tcoe,transaction_date,transaction_year,vintage_year,market_classification,transdate,transyr,primary_secondary_market,spot_or_forward,if_forward_contract_what_percentage_are_options,forward_contract_maturity_date,end_user_or_intermediary,buyer_s_voluntary_goals_details_for_use_of_offsets,if_private_sectore_buyer_department,scopes,_scope_1,_scope_2,_scope_3,if_scope_3_please_describe_activity scope3_activity,did_the_buyer_purchase_the_credits_to_use_internally_or_to_re_sell_for_future_corsia_compliance as buyer_internal_or_resell,if_transaction_is_for_a_compliance_purpose_what_of_their_cap_does_the_buyer_cover_with_this_transaction as buyer_qty_compliance,if_end_buyer_will_buyer_be_retiring_offsets_immediately_or_plan_to_store_offsets_for_future_retirement as buyer_retire_or_store,if_end_buyer_do_they_have_an_internal_policy_regarding_storage_or_retirement_of_vintage_offsets as buyer_retirement_policy,if_yes_what_is_buyer_s_cut_off_year_for_vintage_offsets_to_be_retired as buyer_vintage_cutoff,if_end_buyer_will_buyer_be_integrating_offsets_into_consumer_product_offerings_including_at_point_of_sale_pos as buyer_consumerProduct,what_was_the_cost_per_ton_for_project_development_usd as project_dvpt_usd_ton,how_was_the_project_financed as project_financed_how ,was_organization_s_credit_history_previous_project_development_experience_a_factor_in_securing_project_finance as project_finance_usedCreditHistory,are_the_credits_jurisdictional_scale as buyer_c_jurisdictional,will_the_project_nest_into_a_jurisdictional_redd_program as project_jurisdictional_REDD'
  Filter =' where year_reported is not null or registry_id is not null or volume_transacted_tco2e is not null or volume_w_price is not null or trade_total_value_USD is not null or conversion is not null or price_USD_tcoe is not null or transaction_date is not null or transaction_year is not null or vintage_year is not null or market_classification is not null or transdate is not null or transyr is not null or primary_secondary_market is not null or spot_or_forward is not null or if_forward_contract_what_percentage_are_options is not null or forward_contract_maturity_date is not null or end_user_or_intermediary is not null or buyer_s_voluntary_goals_details_for_use_of_offsets is not null or if_private_sectore_buyer_department is not null or scopes is not null or _scope_1 is not null or _scope_2 is not null or _scope_3 is not null or scope3_activity is not null or buyer_internal_or_resell is not null or buyer_qty_compliance is not null or buyer_retire_or_store is not null or buyer_retirement_policy is not null or buyer_vintage_cutoff is not null or buyer_consumerProduct is not null or project_dvpt_usd_ton is not null or project_financed_how is not null or project_finance_usedCreditHistory is not null or buyer_c_jurisdictional is not null or project_jurisdictional_REDD is not null'
  Query_Alias = '(SELECT '+ source_columns +' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`)'
  Query = 'SELECT * FROM '+Query_Alias+Filter

  
  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')



In [ ]:
def CreateTradesTRN(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):

  source_columns = 'trade_id,projects.project_id,respondents.respondent_id,buyers.buyer_id,year_reported,registry_id,volume_transacted_tco2e,volume_w_price,trade_total_value_USD,conversion,price_USD_tcoe,transaction_date,transaction_year,vintage_year,market_classification,transdate,transyr,primary_secondary_market,spot_or_forward,if_forward_contract_what_percentage_are_options,forward_contract_maturity_date,end_user_or_intermediary,buyer_s_voluntary_goals_details_for_use_of_offsets,if_private_sectore_buyer_department,scopes,_scope_1,_scope_2,_scope_3,scope3_activity,buyer_internal_or_resell,buyer_qty_compliance,buyer_retire_or_store,buyer_retirement_policy,buyer_vintage_cutoff,buyer_consumerProduct,project_dvpt_usd_ton,project_financed_how ,project_finance_usedCreditHistory,buyer_c_jurisdictional,project_jurisdictional_REDD'
  JoiningCondition = '`ecosystemmarketplace.esm_Stage.trades` as trades LEFT JOIN `ecosystemmarketplace.esm_Stage.respondents` as respondents ON respondents.src_Id = trades.trade_id LEFT JOIN `ecosystemmarketplace.esm_Stage.buyers` as buyers ON buyers.src_Id= trades.trade_id LEFT JOIN `ecosystemmarketplace.esm_TRN.projects` as projects ON projects.src_Id= trades.trade_id'
  Query = 'SELECT ' + source_columns +' FROM '+JoiningCondition

  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')


In [ ]:
def CreateTradesFINAL(client,Project_id,SourceDatasetName,SourceTableName,DestinationDatasetName,DestinationTable):

  source_columns = "trade_id,EXTRACT(Year from transyr) trade_year,project_id,respondent_id,buyer_id,year_reported,registry_id,volume_transacted_tco2e,volume_w_price,trade_total_value_USD,conversion,price_USD_tcoe,transaction_date,transaction_year,vintage_year,market_classification,transdate,transyr,CASE WHEN primary_secondary_market ='Primary' or primary_secondary_market='Both' then 1 else 0 end market_IsPrimary ,CASE WHEN primary_secondary_market ='Secondary' or primary_secondary_market='Both' then 1 else 0 end market_IsSecondary,CASE WHEN spot_or_forward ='Spot' or spot_or_forward='Both' then 1 else 0 end market_IsSpot,CASE WHEN spot_or_forward ='Forward' or spot_or_forward='Both' then 1 else 0 end market_IsForward,if_forward_contract_what_percentage_are_options,forward_contract_maturity_date,end_user_or_intermediary,buyer_s_voluntary_goals_details_for_use_of_offsets,if_private_sectore_buyer_department,scopes,_scope_1,_scope_2,_scope_3,scope3_activity,buyer_internal_or_resell,buyer_qty_compliance,buyer_retire_or_store,buyer_retirement_policy,buyer_vintage_cutoff,buyer_consumerProduct,project_dvpt_usd_ton,project_financed_how ,project_finance_usedCreditHistory,buyer_c_jurisdictional,project_jurisdictional_REDD"
  id_column = 'trade_id'
  Query = 'SELECT ' + source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'
  # Every row is already unique no need for LOG.

  df = client.query(Query).to_dataframe()

  df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')



In [ ]:
def CreateTrades(client,Project_id):
  CreateTradesStage(client,Project_id,'esm_Stage','em_src','esm_Stage','trades')
  CreateTradesTRN(client,Project_id,'esm_Stage','trades','esm_TRN','trades')
  CreateTradesFINAL(client,Project_id,'esm_TRN','trades','esm_FINAL','trades')

In [ ]:
# def CreateSdgs():
#   # CREATE TABLE `ecosystemmarketplace.esm_FINAL.SDGS`
#   # (`sdgcode` int NOT NULL,
#   # `sdgname` string
#   # );
#   # INSERT INTO `ecosystemmarketplace.esm_FINAL.SDGS` VALUES
#   # (2,'ZERO HUNGER')
#   # ,(3,'GOOD HEALTH & WELL-BEING')
#   # ,(6,'CLEAN WATER & SANITATION')
#   # ,(7,'AFFORDABLE & CLEAN ENERGY')
#   # ,(8,'DECENT WORK & ECONOMIC GROWTH')
#   # ,(9,'INDUSTRY, INNOVATION & INFRASTRUCTURE')
#   # ,(11,'SUSTAINABLE CITIES & COMMUNITIES')
#   # ,(12,'RESPONSIBLE CONSUMPTION & PRODUCTION')
#   # ,(13,'CLIMATE ACTION')
#   # ,(14,'LIFE BELOW WATER')
#   # ,(15,'LIFE ON LAND')

# Manual ETL

In [ ]:
#@title Default title text
Project_id = "ecosystemmarketplace" #@param {type:"string"}
SourceDatasetName = "em_07May" #@param {type:"string"}
SourceTableName = "08may_08may" #@param {type:"string"}
DestinationDatasetName = "esm_Stage" #@param {type:"string"}
DestinationTable = "em_src" #@param {type:"string"}

# ecosystemmarketplace.em_22Apr.22apr_22apr_cleandate_tableau_output_22apr_cleandate_tableau_output

# Source Datasets
# em_22Apr
# em_06May
# esm_Stage
# em_07May



# Source Tables
# 22apr_22apr_cleandate_tableau_output_22apr_cleandate_tableau_output
# 06may_csv_06may_csv
# em_src
# 08may_08may


# Destination Datasets
# esm_Stage
# esm_TRN





In [ ]:
client = bigquery.Client(project=Project_id)

In [ ]:

# em_src
Query = 'SELECT ROW_NUMBER() OVER (ORDER BY __hevo_id) src_Id,* FROM `'
Query = Query+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'

# countries
# Countries()

#respondents

# creating respondents_Stage
# source_parentId = 'src_Id'
# identifier_column ='respondent_name'
# source_columns = 'RANK() over (Order by trim(em_respondent)) respondent_id , trim(em_respondent) as respondent_name,respondent_role,respondent_profit_status,respondent_website'
# Query = 'SELECT '+source_parentId+',' + source_columns +' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'

# creating respondents_TRN
# source_columns = 'respondents.respondent_id,respondents.respondent_name,respondents.respondent_role,countries.id country_id,respondents.respondent_profit_status,respondents.respondent_website '
# JoiningCondition = '`ecosystemmarketplace.esm_Stage.respondents` as respondents LEFT JOIN `ecosystemmarketplace.esm_Stage.em_src` as src ON respondents.src_Id = src.src_Id LEFT JOIN `ecosystemmarketplace.esm_FINAL.worldcountries` as countries ON src.respondent_country = countries.name'
# Query = 'SELECT ' + source_columns +' FROM '+JoiningCondition


# creating respondents_FINAL

# source_columns = 'respondent_id,respondent_name,country_id,respondent_profit_status,respondent_website'
# JoiningCondition = '(SELECT row_number() over (Partition by respondent_id order by respondent_id) as entry_number,'+source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'` ) '

# Query = 'SELECT ' + source_columns +' FROM '+JoiningCondition+' where entry_number=1'
# LogQuery = 'SELECT ' + source_columns +' FROM '+JoiningCondition+' where entry_number>1'

# creating roles for respondents
# CreateRoles()
# respondentsFinal = 'SELECT respondent_id,respondent_name,respondent_role,country_id,respondent_profit_status,respondent_website FROM `ecosystemmarketplace.esm_FINAL.respondents`' 
# respondentroles = CreateRespondentRoles(client.query(respondentsFinal).to_dataframe())



#creating buyer_Stage
# source_parentId = 'src_Id'
# identifier_column ='buyer_name'
# source_columns = 'RANK() over (Order by trim(buyer_name)) buyer_id , trim(buyer_name) as buyer_name,buyer_profit_status,buyer_profit_status_description,buyer_sector'
# Filter = ' WHERE NULLIF(trim('+identifier_column+'),\'\') IS NOT NULL'
# Query = 'SELECT '+source_parentId+',' + source_columns +' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'+Filter


#creating buyer_TRN
# source_columns = 'buyers.buyer_id,buyers.buyer_name,countries.id country_id,buyers.buyer_profit_status,buyers.buyer_profit_status_description,buyers.buyer_sector'
# JoiningCondition = '`ecosystemmarketplace.esm_Stage.buyers` as buyers LEFT JOIN `ecosystemmarketplace.esm_Stage.em_src` as src ON buyers.src_Id = src.src_Id LEFT JOIN `ecosystemmarketplace.esm_FINAL.worldcountries` as countries ON src.buyer_country = countries.name'
# Query = 'SELECT ' + source_columns +' FROM '+JoiningCondition

#creating buyer_FINAL
# source_columns = 'buyer_id,buyer_name,country_id,buyer_profit_status,buyer_profit_status_description,buyer_sector'
# id_column = 'buyer_id'
# Entry_number = '(SELECT row_number() over (Partition by '+id_column+' order by '+id_column+') as entry_number,'+source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'
# Entry_number_ctd = ') where entry_number'

# Query = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'=1'
# LogQuery = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'>1'



# projects
#creating projects_Stage
# source_parentId = 'src_Id'
# source_columns = 'registry_id,project_name,project_s_type,project_category,reduction_removal,third_party_standard,co_benefits_standard,SDG,corsia_eligibility'
# Filter = ' WHERE project_s_type is not null or project_category is not null or reduction_removal is not null or third_party_standard is not null or co_benefits_standard is not null or SDG is not null or corsia_eligibility  is not null'
# Query = 'SELECT '+source_parentId+',' + source_columns +' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'+Filter



# Project child tables
# CreateProjectCategories(client,Project_id)
# CreateProjectType(client,Project_id)
# CreateProjectCluster(client,Project_id)




#creating projects_TRN
# # MergedVCSBerkeley
# source_merged_VCS ='CASE WHEN coalesce(VCS.project_id,trim(projects.registry_id)) IS NULL THEN CAST(RANK() OVER(ORDER BY projects.src_Id) as string) ELSE coalesce(VCS.project_id,trim(projects.registry_id)) END project_id'
# source_columns = 'projects.src_Id,projects.project_name em_project_name,projects.project_s_type,projects.project_category,projects.reduction_removal em_reduction_removal,projects.third_party_standard,projects.co_benefits_standard,projects.SDG,countries.country_id, projects.corsia_eligibility,VCS.project_name,VCS.voluntary_registry,VCS.arb_project,VCS.voluntary_status,VCS.scope,VCS.type,VCS.reduction_removal,VCS.region,VCS.country,VCS.state,VCS.project_site_location,VCS.project_developer,VCS.total_credits_issued,VCS.total_credits_retired,VCS.total_credits_remaining,VCS.first_year_of_project,VCS.verifier,VCS.registry_arb,VCS.arb_project_detail,VCS.project_type,VCS.registry_documents,VCS.date_added_to_database,VCS.ccb_certifications,VCS.arb_id,VCS.project_website,VCS.project_owner,VCS.offset_project_operator,VCS.authorized_project_designee,VCS.project_listed,VCS.project_registered,VCS.notes_from_registry,VCS.notes_from_berkeley_carbon_trading_project,VCS.methodology_protocol,VCS.estimated_annual_emission_reductions,VCS.pers'
# project_From =' FROM `ecosystemmarketplace.Berkeley_VCM.berkeley_projects_projects` as VCS FULL OUTER JOIN `ecosystemmarketplace.esm_Stage.projects` as projects ON VCS.project_id = trim(projects.registry_id) LEFT JOIN `ecosystemmarketplace.esm_Stage.em_src` as src ON projects.src_Id = src.src_Id LEFT JOIN `ecosystemmarketplace.esm_FINAL.countries` as countries ON src.project_s_country = countries.en'
# Query = 'SELECT '+ source_merged_VCS+','+source_columns +project_From

# source_columns = 'CASE WHEN trim(projects.registry_id) IS NULL THEN CAST(RANK() OVER(ORDER BY projects.src_Id) as string) ELSE trim(projects.registry_id) END project_id,projects.src_Id,projects.project_name em_project_name,projects.project_s_type,projects.project_category,projects.reduction_removal em_reduction_removal,projects.third_party_standard,projects.co_benefits_standard,projects.SDG,countries.id country_id, projects.corsia_eligibility,category.categoryid,emcluster.clusterid,type.typeid'
# project_From =' FROM `ecosystemmarketplace.esm_Stage.projects` as projects LEFT JOIN `ecosystemmarketplace.esm_Stage.em_src` as src ON projects.src_Id = src.src_Id LEFT JOIN `ecosystemmarketplace.esm_FINAL.worldcountries` as countries ON src.project_s_country = countries.name LEFT JOIN `ecosystemmarketplace.esm_FINAL.category` as category ON src.project_category = category.categoryname LEFT JOIN `ecosystemmarketplace.esm_FINAL.cluster` as emcluster ON src.project_cluster = emcluster.clustername LEFT JOIN `ecosystemmarketplace.esm_FINAL.type` as type ON src.project_cluster = type.typename'
# filters = ' where projects.src_Id is not null or projects.project_name is not null or projects.project_s_type is not null or projects.project_category is not null or projects.reduction_removal is not null or projects.third_party_standard is not null or projects.co_benefits_standard is not null or projects.SDG is not null or projects.corsia_eligibility is not null'
# Query = 'SELECT '+source_columns +project_From + filters





#creating project_FINAL
# # MergedVCSBerkeley
# source_columns = 'project_id,em_project_name,project_s_type,project_category,em_reduction_removal,third_party_standard,co_benefits_standard,SDG,country_id, corsia_eligibility,project_name,voluntary_registry,arb_project,voluntary_status,scope,type,reduction_removal,region,country,state,project_site_location,project_developer,total_credits_issued,total_credits_retired,total_credits_remaining,first_year_of_project,verifier,registry_arb,arb_project_detail,project_type,registry_documents,date_added_to_database,ccb_certifications,arb_id,project_website,project_owner,offset_project_operator,authorized_project_designee,project_listed,project_registered,notes_from_registry,notes_from_berkeley_carbon_trading_project,methodology_protocol,estimated_annual_emission_reductions,pers'
# id_column = 'project_id'
# Entry_number = '(SELECT row_number() over (Partition by '+id_column+' order by '+id_column+') as entry_number,'+source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'
# Entry_number_ctd = ') where entry_number'

# Query = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'=1'
# LogQuery = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'>1'

# source_columns = 'project_id,em_project_name,project_s_type,project_category,em_reduction_removal,third_party_standard,co_benefits_standard,SDG,country_id, corsia_eligibility,categoryid,clusterid,typeid'
# id_column = 'project_id'
# Entry_number = '(SELECT row_number() over (Partition by '+id_column+' order by '+id_column+') as entry_number,'+source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'
# Entry_number_ctd = ') where entry_number'

# Query = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'=1'
# LogQuery = 'SELECT ' + source_columns +' FROM '+Entry_number+Entry_number_ctd+'>1'



# creating esmprojectsdgs
# source_columns = 'project_id,SDG'
# filter = " where SDG is not null and SDG NOT LIKE '%UTC%' and SDG not like '%;%' and SDG NOT LIKE '%Not%'"
# Query = 'SELECT ' + source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'+filter



# trade
#creating trade_Stage

# source_columns = 'src_Id trade_id,year_reported,registry_id,volume_transacted_tco2e,volume_w_price,value trade_total_value_USD,conversion,average_price_usd_tco2e price_USD_tcoe,transaction_date,transaction_year,vintage_year,market_classification,transdate,transyr,primary_secondary_market,spot_or_forward,if_forward_contract_what_percentage_are_options,forward_contract_maturity_date,end_user_or_intermediary,buyer_s_voluntary_goals_details_for_use_of_offsets,if_private_sectore_buyer_department,scopes,_scope_1,_scope_2,_scope_3,if_scope_3_please_describe_activity scope3_activity,did_the_buyer_purchase_the_credits_to_use_internally_or_to_re_sell_for_future_corsia_compliance as buyer_internal_or_resell,if_transaction_is_for_a_compliance_purpose_what_of_their_cap_does_the_buyer_cover_with_this_transaction as buyer_qty_compliance,if_end_buyer_will_buyer_be_retiring_offsets_immediately_or_plan_to_store_offsets_for_future_retirement as buyer_retire_or_store,if_end_buyer_do_they_have_an_internal_policy_regarding_storage_or_retirement_of_vintage_offsets as buyer_retirement_policy,if_yes_what_is_buyer_s_cut_off_year_for_vintage_offsets_to_be_retired as buyer_vintage_cutoff,if_end_buyer_will_buyer_be_integrating_offsets_into_consumer_product_offerings_including_at_point_of_sale_pos as buyer_consumerProduct,what_was_the_cost_per_ton_for_project_development_usd as project_dvpt_usd_ton,how_was_the_project_financed as project_financed_how ,was_organization_s_credit_history_previous_project_development_experience_a_factor_in_securing_project_finance as project_finance_usedCreditHistory,are_the_credits_jurisdictional_scale as buyer_c_jurisdictional,will_the_project_nest_into_a_jurisdictional_redd_program as project_jurisdictional_REDD'
# Filter =' where year_reported is not null or registry_id is not null or volume_transacted_tco2e is not null or volume_w_price is not null or trade_total_value_USD is not null or conversion is not null or price_USD_tcoe is not null or transaction_date is not null or transaction_year is not null or vintage_year is not null or market_classification is not null or transdate is not null or transyr is not null or primary_secondary_market is not null or spot_or_forward is not null or if_forward_contract_what_percentage_are_options is not null or forward_contract_maturity_date is not null or end_user_or_intermediary is not null or buyer_s_voluntary_goals_details_for_use_of_offsets is not null or if_private_sectore_buyer_department is not null or scopes is not null or _scope_1 is not null or _scope_2 is not null or _scope_3 is not null or scope3_activity is not null or buyer_internal_or_resell is not null or buyer_qty_compliance is not null or buyer_retire_or_store is not null or buyer_retirement_policy is not null or buyer_vintage_cutoff is not null or buyer_consumerProduct is not null or project_dvpt_usd_ton is not null or project_financed_how is not null or project_finance_usedCreditHistory is not null or buyer_c_jurisdictional is not null or project_jurisdictional_REDD is not null'
# Query_Alias = '(SELECT '+ source_columns +' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`)'
# Query = 'SELECT * FROM '+Query_Alias+Filter

#creating trade_TRN

# source_columns = 'trade_id,projects.project_id,respondents.respondent_id,buyers.buyer_id,year_reported,registry_id,volume_transacted_tco2e,volume_w_price,trade_total_value_USD,conversion,price_USD_tcoe,transaction_date,transaction_year,vintage_year,market_classification,transdate,transyr,primary_secondary_market,spot_or_forward,if_forward_contract_what_percentage_are_options,forward_contract_maturity_date,end_user_or_intermediary,buyer_s_voluntary_goals_details_for_use_of_offsets,if_private_sectore_buyer_department,scopes,_scope_1,_scope_2,_scope_3,scope3_activity,buyer_internal_or_resell,buyer_qty_compliance,buyer_retire_or_store,buyer_retirement_policy,buyer_vintage_cutoff,buyer_consumerProduct,project_dvpt_usd_ton,project_financed_how ,project_finance_usedCreditHistory,buyer_c_jurisdictional,project_jurisdictional_REDD'
# JoiningCondition = '`ecosystemmarketplace.esm_Stage.trades` as trades LEFT JOIN `ecosystemmarketplace.esm_Stage.respondents` as respondents ON respondents.src_Id = trades.trade_id LEFT JOIN `ecosystemmarketplace.esm_Stage.buyers` as buyers ON buyers.src_Id= trades.trade_id LEFT JOIN `ecosystemmarketplace.esm_TRN.projects` as projects ON projects.src_Id= trades.trade_id'
# Query = 'SELECT ' + source_columns +' FROM '+JoiningCondition

#creating trades_FINAL

# source_columns = "trade_id,EXTRACT(Year from transyr) trade_year,project_id,respondent_id,buyer_id,year_reported,registry_id,volume_transacted_tco2e,volume_w_price,trade_total_value_USD,conversion,price_USD_tcoe,transaction_date,transaction_year,vintage_year,market_classification,transdate,transyr,CASE WHEN primary_secondary_market ='Primary' or primary_secondary_market='Both' then 1 else 0 end market_IsPrimary ,CASE WHEN primary_secondary_market ='Secondary' or primary_secondary_market='Both' then 1 else 0 end market_IsSecondary,CASE WHEN spot_or_forward ='Spot' or spot_or_forward='Both' then 1 else 0 end market_IsSpot,CASE WHEN spot_or_forward ='Forward' or spot_or_forward='Both' then 1 else 0 end market_IsForward,if_forward_contract_what_percentage_are_options,forward_contract_maturity_date,end_user_or_intermediary,buyer_s_voluntary_goals_details_for_use_of_offsets,if_private_sectore_buyer_department,scopes,_scope_1,_scope_2,_scope_3,scope3_activity,buyer_internal_or_resell,buyer_qty_compliance,buyer_retire_or_store,buyer_retirement_policy,buyer_vintage_cutoff,buyer_consumerProduct,project_dvpt_usd_ton,project_financed_how ,project_finance_usedCreditHistory,buyer_c_jurisdictional,project_jurisdictional_REDD"
# id_column = 'trade_id'
# Query = 'SELECT ' + source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'
# # Every row is already unique no need for LOG.

#Berkeley Projects
# source_columns ='project_id,project_name,voluntary_registry,arb_project,voluntary_status,scope,type,reduction_removal,region,country,state,project_site_location,project_developer,total_credits_issued,total_credits_retired,total_credits_remaining,first_year_of_project,verifier,registry_arb,arb_project_detail,project_type,registry_documents,date_added_to_database,ccb_certifications,arb_id,project_website,project_owner,offset_project_operator,authorized_project_designee,project_listed,project_registered,notes_from_registry,notes_from_berkeley_carbon_trading_project,methodology_protocol,estimated_annual_emission_reductions,pers'
# Query ='SELECT '+ source_columns+' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'





# #methodology
# sample_query='SELECT DISTINCT methdology_protocol_the_project_adheres_to as methodology_name FROM `'
# sample_query = sample_query+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'


# countries(We have clean table for countries now.)
# source_parentId = 'src_Id'
# source_columns = 'RANK() over (Order by respondent_country) respondent_country_id,respondent_country,RANK() over (Order by project_s_country) project_s_country_id,project_s_country,RANK() over (Order by buyer_country) buyer_country_id,buyer_country'
# Query = 'SELECT '+source_parentId+',' + source_columns +' FROM `'+Project_id+'.'+SourceDatasetName+'.'+SourceTableName +'`'


In [ ]:
print(Query)
# print(LogQuery) # From TRN to FINAL

SELECT ROW_NUMBER() OVER (ORDER BY __hevo_id) src_Id,* FROM `ecosystemmarketplace.em_07May.08may_08may`


In [ ]:
df = client.query(Query).to_dataframe()
# df_Log = client.query(LogQuery).to_dataframe() # From TRN to FINAL

In [ ]:
df.shape

(43646, 68)

In [ ]:
df.columns

Index(['src_Id', 'year_reported', 'em_respondent', 'respondent_role',
       'respondent_country', 'respondent_profit_status', 'respondent_website',
       'projecttype', 'projectcluster', 'reduction_removal',
       'project_category', 'project_s_country', 'project_region',
       'third_party_standard', 'market_classification',
       'average_price_price_tco2e', 'transaction_year', '__hevo_id',
       'registry', 'project_name',
       'methdology_protocol_the_project_adheres_to', 'co_benefits_standard',
       'additional_certification_s', 'sdg_assessed', 'sdg',
       'market_s_in_which_project_is_eligible', 'volume_transacted_tco2e',
       'volume_w_price', 'currency', 'conversion', 'average_price_usd_tco2e',
       'value', 'transaction_date', 'vintage_year', 'primary_secondary_market',
       'spot_or_forward', 'buyer_name', 'buyer_country', 'buyer_region',
       'buyer_profit_status', 'end_user_or_intermediary',
       'buyer_s_voluntary_goals_details_for_use',
       'if_pr

In [ ]:
renamed_columns={'src_Id'	:	'src_Id'
 ,'year_reported'	:	 'year_reported'
 ,'em_respondent'	:	 'em_respondent'
 ,'respondent_role'	:	 'respondent_role'
 ,'respondent_country'	:	 'respondent_country'
 ,'respondent_profit_status'	:	 'respondent_profit_status'
 ,'respondent_website'	:	 'respondent_website'
 ,'market_classification'	:	 'market_classification'
 ,'conversion'	:	 'conversion'
 ,'transaction_year'	:	 'transaction_year'
 ,'transdate'	:	 'transdate'
 ,'transyr'	:	 'transyr'
 ,'original_order'	:	 'original_order'
 ,'duplicate_row'	:	 'duplicate_row'
 ,'__hevo_id'	:	 '__hevo_id'
 ,'__hevo__ingested_at'	:	 '__hevo__ingested_at'
 ,'__hevo__loaded_at'	:	 '__hevo__loaded_at'
 ,'project_type'	:	 'project_s_type'
 ,'project_cluster'	:	 'project_cluster'
 ,'reduction_removal'	:	 'reduction_removal'
 ,'project_category'	:	 'project_category'
 ,'project_s_country'	:	 'project_s_country'
 ,'project_region'	:	 'project_region'
 ,'third_party_standard'	:	 'third_party_standard'
 ,'market_s_in_which_project_is_eligible'	:	 'market_s_in_which_project_is_eligible'
 ,'volume_transacted_tco2e'	:	 'volume_transacted_tco2e'
 ,'volume_w_price'	:	 'volume_w_price'
 ,'currency'	:	 'currency'
 ,'average_price_usd_tco2e'	:	 'average_price_usd_tco2e'
 ,'value'	:	 'value'
 ,'vintage_year'	:	 'vintage_year'
 ,'primary_secondary_market'	:	 'primary_secondary_market'
 ,'spot_or_forward'	:	 'spot_or_forward'
 ,'if_private_sectore_buyer_department'	:	 'if_private_sectore_buyer_department'
 ,'if_scope_3_please_describe_activity'	:	 'if_scope_3_please_describe_activity'
 ,'if_end_buyer_will_buyer_be_integrating_offsets_into_consumer_product_offerings_including_at_point_of_sale_pos'	:	 'if_end_buyer_will_buyer_be_integrating_offsets_into_consumer_product_offerings_including_at_point_of_sale_pos'
 ,'will_the_project_nest_into_a_jurisdictional_redd_program'	:	 'will_the_project_nest_into_a_jurisdictional_redd_program'
 ,'co_benefits_standard'	:	 'co_benefits_standard'
 ,'additional_certification_s'	:	 'additional_certification_s'
 ,'end_user_or_intermediary'	:	 'end_user_or_intermediary'
 ,'registry'	:	 'registry_id'
 ,'project_name'	:	 'project_name'
 ,'if_forward_contract_what_percentage_are_options'	:	 'if_forward_contract_what_percentage_are_options'
 ,'methdology_protocol_the_project_adheres_to'	:	 'methdology_protocol_the_project_adheres_to'
 ,'corsia_eligibility'	:	 'corsia_eligibility'
 ,'buyer_name'	:	 'buyer_name'
 ,'buyer_country'	:	 'buyer_country'
 ,'buyer_region'	:	 'buyer_region'
 ,'buyer_profit_status'	:	 'buyer_profit_status'
 ,'buyer_profit_status_description'	:	 'buyer_profit_status_description'
 ,'buyer_sector'	:	 'buyer_sector'
 ,'if_end_buyer_will_buyer_be_retiring_offsets_immediately_or_plan_to_store_offsets_for_future_retirement'	:	 'if_end_buyer_will_buyer_be_retiring_offsets_immediately_or_plan_to_store_offsets_for_future_retirement'
 ,'sdg_assessed'	:	 'sdg_assessed'
 ,'sdg'	:	 'sdg'
 ,'buyer_s_voluntary_goals_details_for_use'	:	 'buyer_s_voluntary_goals_details_for_use_of_offsets'
 ,'_scope_2'	:	 '_scope_2'
 ,'_scope_3'	:	 '_scope_3'
 ,'did_the_buyer_purchase_the_credits_to_use_internally_or_to_re_sell_for_future_corsia_compliance'	:	 'did_the_buyer_purchase_the_credits_to_use_internally_or_to_re_sell_for_future_corsia_compliance'
 ,'if_end_buyer_do_they_have_an_internal_policy_regarding_storage_or_retirement_of_vintage_offsets'	:	 'if_end_buyer_do_they_have_an_internal_policy_regarding_storage_or_retirement_of_vintage_offsets'
 ,'if_yes_what_is_buyer_s_cut_off_year_for_vintage_offsets_to_be_retired'	:	 'if_yes_what_is_buyer_s_cut_off_year_for_vintage_offsets_to_be_retired'
 ,'what_was_the_cost_per_ton_for_project_development_usd'	:	 'what_was_the_cost_per_ton_for_project_development_usd'
 ,'how_was_the_project_financed'	:	 'how_was_the_project_financed'
 ,'was_organization_s_credit_history_previous_project_development_experience_a_factor_in_securing_project_finance'	:	 'was_organization_s_credit_history_previous_project_development_experience_a_factor_in_securing_project_finance'
 ,'are_the_credits_jurisdictional_scale'	:	 'are_the_credits_jurisdictional_scale'
 ,'scopes'	:	 'scopes'
 ,'date_last_modified'	:	 'date_last_modified'
 ,'transaction_other_info'	:	 'transaction_other_info'
 ,'if_transaction_is_for_a_compliance_purpose_what_of_their_cap_does_the_buyer_cover_with_this_transaction'	:	 'if_transaction_is_for_a_compliance_purpose_what_of_their_cap_does_the_buyer_cover_with_this_transaction'
 ,'transyear'	:	 'transyear'
 ,'average_price_price_tco2e'	:	 'average_price_price_tco2e'
 ,'transaction_date'	:	 'transaction_date'
 ,'forward_contract_maturity_date'	:	 'forward_contract_maturity_date'
 ,'date_of_input'	:	 'date_of_input'
 ,'_scope_1'	:	 '_scope_1'
 ,'__hevo__marked_deleted'	:	 '__hevo__marked_deleted'
}
df = df.rename(columns=renamed_columns)

In [ ]:
df.dtypes

src_Id                                                                                                                          int64
year_reported                                                                                                     datetime64[ns, UTC]
em_respondent                                                                                                                  object
respondent_role                                                                                                                object
respondent_country                                                                                                             object
respondent_profit_status                                                                                                       object
respondent_website                                                                                                             object
projecttype                                                   

In [ ]:
# df['country_id'] = df['country_id'].replace(np.NaN,0).astype('int64')
# df['categoryid'] = df['categoryid'].replace(np.NaN,0).astype('int64')
# df['clusterid'] = df['clusterid'].replace(np.NaN,0).astype('int64')
# df['typeid'] = df['typeid'].replace(np.NaN,0).astype('int64')

# Working for respondentroles

In [ ]:
respondentroles.head()

,respondent_id,roleid
0,2616,5
1,2616,2
2,2616,1
3,2616,3
4,483,5


In [ ]:
respondentroles.to_gbq(project_id=Project_id,destination_table='esm_FINAL.respondentroles',if_exists='replace')

# Working for SDGs

In [ ]:
df.head()

,trade_id,trade_year,project_id,respondent_id,buyer_id,year_reported,registry_id,volume_transacted_tco2e,volume_w_price,trade_total_value_USD,conversion,price_USD_tcoe,transaction_date,transaction_year,vintage_year,market_classification,transdate,transyr,market_IsPrimary,market_IsSecondary,market_IsSpot,market_IsForward,if_forward_contract_what_percentage_are_options,forward_contract_maturity_date,end_user_or_intermediary,buyer_s_voluntary_goals_details_for_use_of_offsets,if_private_sectore_buyer_department,scopes,_scope_1,_scope_2,_scope_3,scope3_activity,buyer_internal_or_resell,buyer_qty_compliance,buyer_retire_or_store,buyer_retirement_policy,buyer_vintage_cutoff,buyer_consumerProduct,project_dvpt_usd_ton,project_financed_how,project_finance_usedCreditHistory,buyer_c_jurisdictional,project_jurisdictional_REDD
0,2989,2020.0,GS5099,2507,NaN,2021-01-01T00:00:00Z,GS5099,10015.0,100000.0,292745.51,1.12,29.23,None,2020-01-01 00:00:00+00:00,2017.0,Voluntary Carbon Market,2020-01-01T00:00:00Z,2020-01-01 00:00:00+00:00,1,0,1,0,None,None,Both,None,Oil production,None,None,NaN,0.0,Event,Yes,None,Immediately,No,None,No,None,Working capital,No,No,No
1,3706,2020.0,PV100000000000000,3739,272.0,2021-01-01T00:00:00Z,PV100000000000000,4.0,4.0,120.00,1.00,30.00,None,2020-01-01 00:00:00+00:00,2010.0,Voluntary Carbon Market,2020-01-01T00:00:00Z,2020-01-01 00:00:00+00:00,1,0,1,0,None,None,End User,Voluntary carbon neutral goal,Marketing,None,None,NaN,1.0,N/A,None,None,Immediately,None,None,N/A,None,None,None,None,NA
2,3971,2020.0,VCS872,3739,272.0,2021-01-01T00:00:00Z,VCS872,400.0,400.0,12000.00,1.00,30.00,None,2020-01-01 00:00:00+00:00,2004.0,Voluntary Carbon Market,2020-01-01T00:00:00Z,2020-01-01 00:00:00+00:00,1,0,1,0,None,None,End User,Voluntary carbon neutral goal,Public Relations,None,None,NaN,1.0,N/A,None,None,Immediately,None,None,N/A,None,None,None,None,NA
3,3972,2020.0,VCS872,3739,272.0,2021-01-01T00:00:00Z,VCS872,290.0,290.0,8700.00,1.00,30.00,None,2020-01-01 00:00:00+00:00,2006.0,Voluntary Carbon Market,2020-01-01T00:00:00Z,2020-01-01 00:00:00+00:00,1,0,1,0,None,None,End User,Voluntary carbon neutral goal,Public Relations,None,None,NaN,1.0,N/A,None,None,Immediately,None,None,N/A,None,None,None,None,NA
4,3973,2020.0,VCS872,3739,111.0,2021-01-01T00:00:00Z,VCS872,128.0,128.0,3840.00,1.00,30.00,None,2020-01-01 00:00:00+00:00,2006.0,Voluntary Carbon Market,2020-01-01T00:00:00Z,2020-01-01 00:00:00+00:00,1,0,1,0,None,None,End User,Voluntary carbon neutral goal,Public Relations,None,None,NaN,1.0,N/A,None,None,Immediately,None,None,N/A,None,None,None,None,NA


In [ ]:
# BerkeleySdgs
# SDGS_mapping={'ZERO HUNGER':2
# ,'GOOD HEALTH & WELL-BEING':3
# ,'CLEAN WATER & SANITATION':6
# ,'AFFORDABLE & CLEAN ENERGY':7
# ,'DECENT WORK & ECONOMIC GROWTH':8
# ,'INDUSTRY, INNOVATION & INFRASTRUCTURE':9
# ,'SUSTAINABLE CITIES & COMMUNITIES':11
# ,'RESPONSIBLE CONSUMPTION & PRODUCTION':12
# ,'CLIMATE ACTION':13
# ,'LIFE BELOW WATER':14
# ,'LIFE ON LAND':15
# ,'UNDER VALIDATION':None
# ,'UNDER VALIDATION AND VERIFICATION':None
# ,'UNDER VERIFICATION':None
# ,'VALIDATION APPROVED':None
# ,'VERIFICATION APPROVAL REQUESTED':None
# ,'VERIFICATION APPROVED':None

# }
# def convertToSDG(SDGNamelist):
#   sdgs = []
#   if SDGNamelist is None:
#     return None
#   else:
#     for item in SDGNamelist:
#       sdgs.append(SDGS_mapping[item.strip().upper()])
#   return sdgs


In [ ]:
# SDGS esm_FINAL.projects 
df_sdgs = df[['project_id','SDG']].copy()
df_sdgs['SDG'] = df_sdgs['SDG'].str.strip().str.replace(', ',',').str.replace(' ',',').str.split(',')

# # BerkeleySdgs
# # Sdgs
# df_sdgs = df[['project_id','ccb_certifications']].copy()
# df_sdgs['ccb_certifications'] = df['ccb_certifications'].str.split(';')
# df_sdgs['ccb_certifications'] = df_sdgs['ccb_certifications'].apply(convertToSDG)

In [ ]:
project_list=[]
sdgs_list=[]
invalid_project_list = []
invalid_sdgs_list = []
for rownum,row in enumerate(df_sdgs.iterrows()):
  project = row[1]['project_id']
  
  for index,sdgvalue in enumerate(row[1].explode()):
    if index > 0:
      if float(sdgvalue)<=17:
        project_list.append(project)
        sdgs_list.append(int(float(sdgvalue)))
      else:
        invalid_project_list.append(project)
        invalid_sdgs_list.append(sdgvalue)          
projectsdgs_merged = list(zip(project_list,sdgs_list))
projectsdgs = pd.DataFrame(projectsdgs_merged, columns = ['project_id', 'sdgs'])

invalidprojectsdgs_merged = list(zip(invalid_project_list,invalid_sdgs_list))
invalidprojectsdgs = pd.DataFrame(invalidprojectsdgs_merged, columns = ['project_id', 'sdgs'])


In [ ]:
projectsdgs['sdgs'].unique()

array([ 7, 13,  8,  5,  3, 15,  1,  4,  2,  6,  9, 11, 14, 12, 17, 10, 16])

In [ ]:
projectsdgs.shape

(5638, 2)

In [ ]:
# projectsdgs.dtypes
projectsdgs = projectsdgs.rename(columns={'SDG':'sdgs'})
# projectsdgs['sdgs'] = projectsdgs['sdgs'].astype('int64')

In [ ]:
print(df.shape)
# print(df_Log.shape)  # From TRN to FINAL

(5890, 36)


In [ ]:
df = df.rename(columns={'ccb_certifications':'sdgs'})
df.dtypes
# df['trade_year'] = df['trade_year'].replace(np.NAN,0).astype('datetime64') # execute if needed
# (Keep nulls in visuals) # df['trade_year'] = df['trade_year'].replace(np.NAN,0).astype('int64') # execute if needed
# df['country_id'] = df['country_id'].replace(np.NAN,0).astype('int64') # execute if needed

project_id                                                 object
project_name                                               object
voluntary_registry                                         object
arb_project                                                object
voluntary_status                                           object
scope                                                      object
type                                                       object
reduction_removal                                          object
region                                                     object
country                                                    object
state                                                      object
project_site_location                                      object
project_developer                                          object
total_credits_issued                                      float64
total_credits_retired                                     float64
total_cred

# ETL Contd.

In [ ]:
df.describe(include='all')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,trade_id,project_id,respondent_id,buyer_id,year_reported,registry_id,volume_transacted_tco2e,volume_w_price,trade_total_value_USD,conversion,price_USD_tcoe,transaction_date,transaction_year,vintage_year,market_classification,transdate,transyr,primary_secondary_market,spot_or_forward,if_forward_contract_what_percentage_are_options,forward_contract_maturity_date,end_user_or_intermediary,buyer_s_voluntary_goals_details_for_use_of_offsets,if_private_sectore_buyer_department,scopes,_scope_1,_scope_2,_scope_3,scope3_activity,buyer_internal_or_resell,buyer_qty_compliance,buyer_retire_or_store,buyer_retirement_policy,buyer_vintage_cutoff,buyer_consumerProduct,project_dvpt_usd_ton,project_financed_how,project_finance_usedCreditHistory,buyer_c_jurisdictional,project_jurisdictional_REDD
count,4091.000000,3965,4091.000000,337.000000,4075,751,3.947000e+03,3.663000e+03,3.730000e+03,4001.000000,3731.000000,5,4070,2395,4047,4069,4083,1981,1920,115,49,2467,447,3355,443,17,19.000000,16.000000,3271,144,8,238,93,2,686,42,132,108,112,706
unique,NaN,3404,NaN,NaN,5,190,NaN,NaN,NaN,NaN,NaN,3,9,98,7,11,9,2,3,9,23,5,4,11,7,5,NaN,NaN,17,5,2,2,3,1,4,18,4,2,3,6
top,NaN,VCS872,NaN,NaN,2020-01-01T00:00:00Z,VCS872,NaN,NaN,NaN,NaN,NaN,2021-10-01 00:00:00.000,2019-01-01 00:00:00+00:00,2017.0,Voluntary Carbon Market,2019-01-01T00:00:00Z,2019-01-01 00:00:00+00:00,Primary,Spot,N/A,2022-09-01 00:00:00.000,End User,Voluntary goal unknown,NA,Scope 1,100.0,NaN,NaN,N/A,No,Yes,Immediately,No,2041.0,N/A,1.0,Working capital,Yes,No,NA
freq,NaN,163,NaN,NaN,1550,163,NaN,NaN,NaN,NaN,NaN,3,1532,370,3629,1532,1532,1100,1433,59,21,1979,218,2750,344,5,NaN,NaN,3214,83,5,212,78,2,578,14,39,82,78,593
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-01-01 00:00:00+00:00,NaN,NaN,NaN,2012-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-01 00:00:00+00:00,NaN,NaN,NaN,2022-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2046.000000,NaN,1909.907602,155.940653,NaN,NaN,1.226955e+05,1.233500e+05,4.964083e+05,7.942537,8.109573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.410526,10.075000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1181.114304,NaN,1212.745049,87.816245,NaN,NaN,1.344635e+06,1.389530e+06,5.603782e+06,154.025243,13.603088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.258004,26.939939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,1.000000,1.000000,NaN,NaN,1.000000e+00,1.000000e+00,4.430000e+00,0.000000,0.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1023.500000,NaN,997.000000,79.000000,NaN,NaN,4.500000e+02,5.000000e+02,2.440000e+03,1.000000,2.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.875000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df_Log.describe(include='all')  # From TRN to FINAL

In [ ]:
df.isnull().sum()

trade_id                                                 0
year_reported                                           16
registry_id                                           3340
volume_transacted_tco2e                                144
volume_w_price                                         428
trade_total_value_USD                                  361
conversion                                              90
price_USD_tcoe                                         360
transaction_date                                      4086
transaction_year                                        21
vintage_year                                          1696
market_classification                                   44
transdate                                               22
transyr                                                  8
primary_secondary_market                              2110
spot_or_forward                                       2171
if_forward_contract_what_percentage_are_options       39

In [ ]:
# df_Log.isnull().sum()  # From TRN to FINAL

In [ ]:
print(df.shape)
# print(df_Log.shape)

(4091, 37)


In [ ]:
df.head()

,trade_id,project_id,respondent_id,buyer_id,year_reported,registry_id,volume_transacted_tco2e,volume_w_price,trade_total_value_USD,conversion,price_USD_tcoe,transaction_date,transaction_year,vintage_year,market_classification,transdate,transyr,primary_secondary_market,spot_or_forward,if_forward_contract_what_percentage_are_options,forward_contract_maturity_date,end_user_or_intermediary,buyer_s_voluntary_goals_details_for_use_of_offsets,if_private_sectore_buyer_department,scopes,_scope_1,_scope_2,_scope_3,scope3_activity,buyer_internal_or_resell,buyer_qty_compliance,buyer_retire_or_store,buyer_retirement_policy,buyer_vintage_cutoff,buyer_consumerProduct,project_dvpt_usd_ton,project_financed_how,project_finance_usedCreditHistory,buyer_c_jurisdictional,project_jurisdictional_REDD
0,3819,VCS868,994,NaN,2021-01-01T00:00:00Z,VCS868,1300000.0,1300000.0,936000.00,1.00,0.72,None,2020-01-01 00:00:00+00:00,2010-2012,Voluntary Carbon Market,2020-01-01T00:00:00Z,2020-01-01 00:00:00+00:00,None,None,None,None,None,None,NA,None,None,NaN,NaN,N/A,None,None,None,None,None,N/A,1500000 year,None,Yes,None,Yes
1,3820,VCS868,994,NaN,2021-01-01T00:00:00Z,VCS868,488000.0,488000.0,1595760.00,1.00,3.27,None,2020-01-01 00:00:00+00:00,2013-2016,Voluntary Carbon Market,2020-01-01T00:00:00Z,2020-01-01 00:00:00+00:00,None,None,None,None,None,None,NA,None,None,NaN,NaN,N/A,None,None,None,None,None,N/A,1500000 year,None,Yes,None,Yes
2,3024,GS769,3436,NaN,2022-01-01T00:00:00Z,GS769,219578.0,219578.0,677990.99,1.12,3.09,None,2020-01-01 00:00:00+00:00,2021.0,Voluntary Carbon Market,2020-01-01T00:00:00Z,2020-01-01 00:00:00+00:00,Primary,Spot,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None,Cannot disclose,None,Prefer not to say,Prefer not to say
3,3036,None,1619,NaN,2022-01-01T00:00:00Z,None,NaN,NaN,NaN,NaN,NaN,None,NaT,None,None,None,NaT,None,None,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None
4,3037,2918,1619,NaN,2022-01-01T00:00:00Z,None,NaN,NaN,NaN,NaN,NaN,None,NaT,None,None,None,NaT,None,None,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
# df_Log.head()

,project_id,em_project_name,project_s_type,project_category,em_reduction_removal,third_party_standard,co_benefits_standard,SDG,country_id,corsia_eligibility,categoryid,clusterid,typeid
0,GS6395,Wind Power Project in Madhya Pradesh by OBWPPL,Biogas,Renewable Energy,Reduction,Gold Standard,None,"7, 13, 8, 5, 3, 15, 1, 4, 2",101,yes,7,4,4
1,PV_2002_002,Trees for global benefits,Afforestation/reforestation,Forestry and Land Use,Removal,Plan Vivo,None,None,229,None,4,1,0
2,PV_2002_002,Trees for global benefits,Afforestation/reforestation,Forestry and Land Use,Removal,Plan Vivo,None,None,229,None,4,1,0
3,PV_2002_002,Trees for global benefits,Afforestation/reforestation,Forestry and Land Use,Removal,Plan Vivo,None,None,229,None,4,1,0
4,PV_2016_015,"Pastures, convervation, climate action",Grassland/rangeland management,Agriculture,Both,Plan Vivo,None,None,146,None,1,17,16


In [ ]:
print(DestinationDatasetName+'.'+DestinationTable)
# print(DestinationDatasetName+'.Log_'+DestinationTable)  # From TRN to FINAL

esm_Stage.em_src


# Saving SDGs

In [ ]:
invalidprojectsdgs.to_gbq(project_id=Project_id,destination_table='esm_Stage.invalid_esmprojectsdgs',if_exists='replace',location='europe-north1')
projectsdgs.to_gbq(project_id=Project_id,destination_table='esm_FINAL.esmprojectsdgs',if_exists='replace',location='europe-north1')
# projectsdgs.to_gbq(project_id=Project_id,destination_table='esm_FINAL.projectsdgs',if_exists='replace')

1it [00:07,  7.74s/it]
1it [00:08,  8.03s/it]


# ETL Contd.

In [ ]:
df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace',location='europe-north1')
# df_Log.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.Log_'+DestinationTable,if_exists='replace',location='europe-north1')  # From TRN to FINAL

1it [00:35, 35.78s/it]


# WeightedAvg Tables

In [ ]:

Project_id = "ecosystemmarketplace" 
DestinationDatasetName = "esm_FINAL"
DestinationTable = "weighted_Year" 

client = bigquery.Client(project=Project_id)


Standard_Query = '''
SELECT Standard
        ,Year
        ,sum(Volume) TotalVolume
        ,round(SUM(value)/SUM(volume),2) AvgPrice
        ,SUM(Value) TotalValue
from 
(
SELECT src.third_party_standard Standard
        ,EXTRACT(Year from src.transyr) Year
        ,src.volume_transacted_tco2e Volume
        ,coalesce(src.average_price_usd_tco2e,WeightedAvgTable.WeightedAvg) Price
       ,(round(coalesce(src.average_price_usd_tco2e,WeightedAvgTable.WeightedAvg),2) * volume_transacted_tco2e) Value
FROM
(SELECT third_party_standard
        ,EXTRACT(Year from transyr) Year
        ,SUM(volume_transacted_tco2e * average_price_usd_tco2e)/SUM(volume_transacted_tco2e) WeightedAvg
FROM `ecosystemmarketplace.esm_Stage.em_src`
WHERE volume_transacted_tco2e is not null
      and average_price_usd_tco2e is not null 
      and third_party_standard is not null
group by third_party_standard , EXTRACT(Year from transyr) 
)WeightedAvgTable
inner JOIN `ecosystemmarketplace.esm_Stage.em_src` src ON src.third_party_standard = WeightedAvgTable.third_party_standard and EXTRACT(Year from src.transyr) =WeightedAvgTable.Year
where src.volume_transacted_tco2e is not null
order by src.third_party_standard
)
group by Standard
        ,Year
'''

Country_Query = '''
SELECT Country
        ,Year
        ,sum(Volume) TotalVolume
        ,round(SUM(value)/SUM(volume),2) AvgPrice
        ,SUM(Value) TotalValue
from 
(
SELECT src.project_s_country Country
        ,EXTRACT(Year from src.transyr) Year
        ,src.volume_transacted_tco2e Volume
        ,coalesce(src.average_price_usd_tco2e,WeightedAvgTable.WeightedAvg) Price
        ,(round(coalesce(src.average_price_usd_tco2e,WeightedAvgTable.WeightedAvg),2) * volume_transacted_tco2e) Value
FROM
(SELECT project_s_country
        ,EXTRACT(Year from transyr) Year
        ,SUM(volume_transacted_tco2e * average_price_usd_tco2e)/SUM(volume_transacted_tco2e) WeightedAvg
FROM `ecosystemmarketplace.esm_Stage.em_src`
WHERE volume_transacted_tco2e is not null
      and average_price_usd_tco2e is not null 
      and project_s_country is not null
group by project_s_country , EXTRACT(Year from transyr) 
)WeightedAvgTable
inner JOIN `ecosystemmarketplace.esm_Stage.em_src` src ON src.project_s_country = WeightedAvgTable.project_s_country and EXTRACT(Year from src.transyr) =WeightedAvgTable.Year
where src.volume_transacted_tco2e is not null
order by src.project_s_country
)
group by Country
        ,Year

'''
Category_query='''
SELECT ProjectCategory
        ,Year
        ,sum(Volume) TotalVolume
        ,round(SUM(value)/SUM(volume),2) AvgPrice
        ,SUM(Value) TotalValue
from 
(
SELECT src.project_category ProjectCategory
        ,EXTRACT(Year from src.transyr) Year
        ,src.volume_transacted_tco2e Volume
        ,coalesce(src.average_price_usd_tco2e,WeightedAvgTable.WeightedAvg) Price
        ,(round(coalesce(src.average_price_usd_tco2e,WeightedAvgTable.WeightedAvg),2) * volume_transacted_tco2e) Value
FROM
(SELECT project_category
        ,EXTRACT(Year from transyr) Year
        ,SUM(volume_transacted_tco2e * average_price_usd_tco2e)/SUM(volume_transacted_tco2e) WeightedAvg
FROM `ecosystemmarketplace.esm_Stage.em_src`
WHERE volume_transacted_tco2e is not null
      and average_price_usd_tco2e is not null 
      and project_category is not null
group by project_category , EXTRACT(Year from transyr)
)WeightedAvgTable
inner JOIN `ecosystemmarketplace.esm_Stage.em_src` src ON src.project_category = WeightedAvgTable.project_category 
                                                        and EXTRACT(Year from src.transyr) =WeightedAvgTable.Year
where src.volume_transacted_tco2e is not null
order by src.project_category
)
group by ProjectCategory
        ,Year
order by ProjectCategory,Year desc

'''
Year_Query ='''
SELECT Year
        ,sum(Volume) TotalVolume
        ,round(SUM(value)/SUM(volume),2) AvgPrice
        ,SUM(Value) TotalValue
from 
(
SELECT EXTRACT(Year from src.transyr) Year
        ,src.volume_transacted_tco2e Volume
        ,coalesce(src.average_price_usd_tco2e,WeightedAvgTable.WeightedAvg) Price
        ,(round(coalesce(src.average_price_usd_tco2e,WeightedAvgTable.WeightedAvg),2) * volume_transacted_tco2e) Value
FROM
(SELECT EXTRACT(Year from transyr) Year
        ,SUM(volume_transacted_tco2e * average_price_usd_tco2e)/SUM(volume_transacted_tco2e) WeightedAvg
FROM `ecosystemmarketplace.esm_Stage.em_src`
WHERE volume_transacted_tco2e is not null
      and average_price_usd_tco2e is not null 
      and transyr is not null
group by EXTRACT(Year from transyr) 
)WeightedAvgTable
inner JOIN `ecosystemmarketplace.esm_Stage.em_src` src ON EXTRACT(Year from src.transyr) =WeightedAvgTable.Year
where src.volume_transacted_tco2e is not null
order by src.transyr
)
group by Year

'''
df = client.query(Year_Query).to_dataframe()


In [ ]:
df.head()

,Year,TotalVolume,AvgPrice,TotalValue
0,2014,2448481.0,2.83,6.929201e+06
1,2020,227075943.0,3.20,7.274701e+08
2,2021,659318149.0,4.34,2.858233e+09
3,2018,125177352.0,3.58,4.476961e+08
4,2022,16987679.0,8.36,1.420943e+08


In [ ]:
df.dtypes

Year             int64
TotalVolume    float64
AvgPrice       float64
TotalValue     float64
dtype: object

In [ ]:
df.to_gbq(project_id=Project_id,destination_table=DestinationDatasetName+'.'+DestinationTable,if_exists='replace')

1it [00:05,  5.06s/it]
